# First Exercise for Pattern Recognition

**Goal:** Implement a KNN (K nearest neighbour) classification algorithm from scratch

## Overview of the approach
1. Read the data from the MNIST Dataset
2. Write KNN Classification Algorithm
4. Run the classification function over the train set to parametrise the mode
5. Test the outcome with the test set

Import needed packages

In [28]:
import numpy
from math import sqrt, pow

Define Reading function for file paths

In [29]:
def readMnistData(filePath):
    return numpy.genfromtxt(filePath, delimiter=",")

In [30]:
def getClassification(mnistDataEntry: numpy.ndarray):
    return mnistDataEntry[0]

def getMnistData(mnistDataEntry: numpy.ndarray):
    return numpy.delete(mnistDataEntry, [0])

Define finding the euclidean distance

In [31]:
def euclideanDistance(vectorA: numpy.ndarray, vectorB: numpy.ndarray):
    if vectorA.shape != vectorB.shape:
        raise Exception("Vectors must have the same shape")
    else:
        sumOfSquares = 0
        for currentIndex in range(0, len(vectorA)):
            sumOfSquares += pow(vectorA[currentIndex] - vectorB[currentIndex], 2)
        return sqrt(sumOfSquares)

Read the train file

In [32]:
trainDataSet = readMnistData("train.csv")
testDataSet = readMnistData("test.csv")

First try to brute force one classification

## Brute force
Here follows the first try with just a brute force approach with no optimizations. The programm runs extremely slow.

In [36]:
def bruteforce1nn(toClassify: numpy.ndarray, trainSet: numpy.ndarray):
    withoutClassification = getMnistData(toClassify)
    smallestDistance = 100000
    bestCurrentClassifier = -1
    for currentIndex in range(0, len(trainSet)):
        currentTrainData = trainDataSet[currentIndex]
        currentDistance = euclideanDistance(getMnistData(currentTrainData), withoutClassification)
        if smallestDistance > currentDistance:
            smallestDistance = currentDistance
            bestCurrentClassifier = getClassification(currentTrainData)
    return bestCurrentClassifier


def checkClassification(classification, toClassify: numpy.ndarray):
    return classification == getClassification(toClassify)

In [39]:
correct = 0
faulty = 0
for currentDataSet in testDataSet:
    classification = bruteforce1nn(currentDataSet, trainDataSet)
    isCorrect = checkClassification(classification, currentDataSet)
    if isCorrect:
        correct+= 1
        print("Correct")
    else:
        faulty+= 1
        print("Faulty")
print("correct: " + correct)
print("Not Correct " + faulty)

Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct


KeyboardInterrupt: 